# LangChain Basics - Simple Chat Implementation

This notebook demonstrates simple chat functionality with LangChain.

## Features:
- Prompt Template
- Simple conversation with memory
- Streaming and non-streaming options
- Interactive chat interface


In [19]:
%pip install langchain --quiet
%pip install openai --quiet
%pip install langchain_community --quiet
%pip install -U langchain_openai --quiet



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [21]:
from langchain import LLMChain, OpenAI, PromptTemplate


In [22]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.7)
text = "What is the capital of France?"
response = llm.invoke(text)
print(response)




The capital of France is Paris.


In [23]:
# Create a simple prompt template
template = "Tell me a {adjective} joke about {topic}."
prompt = PromptTemplate(
    input_variables=["adjective", "topic"],
    template=template,
)

# Use the prompt template
result = prompt.format(adjective="funny", topic="programming")
print(result)

print("OpenAI response:", llm.invoke(result))


Tell me a funny joke about programming.
OpenAI response: 

Why did the programmer quit his job?

Because he didn't get arrays!


In [ ]:

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage

# Initialize the chat model
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

# Simple conversation with manual memory
conversation_history = []

def chat(message):
    # Add user message to history
    conversation_history.append(HumanMessage(content=message))
    
    # Get response from the model
    response = llm.invoke(conversation_history)
    
    # Add AI response to history
    conversation_history.append(response)
    
    return response.content

# Have a conversation
print("User: Hi, my name is Alice.")
print("Bot:", chat("Hi, my name is Alice."))

print("\nUser: What's my name?")
print("Bot:", chat("What's my name?"))

print("\nUser: What have we talked about so far?")
print("Bot:", chat("What have we talked about so far?"))

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
import time

# Initialize the chat model with optimized settings
llm = ChatOpenAI(
    temperature=0.7, 
    model="gpt-4o-mini",
    max_tokens=500,  # Limit response length for efficiency
    streaming=True  # Enable streaming capability
)

# Optimized conversation with memory management
conversation_history = []
MAX_HISTORY = 10  # Keep only last 10 messages to manage context

def chat(message, stream=False):
    # Add user message to history
    conversation_history.append(HumanMessage(content=message))
    
    # Trim history if it gets too long (memory management)
    if len(conversation_history) > MAX_HISTORY:
        conversation_history[:] = conversation_history[-MAX_HISTORY:]
    
    if stream:
        # Streaming response - shows text appearing in real-time
        print("Bot: ", end="", flush=True)
        full_response = ""
        
        for chunk in llm.stream(conversation_history):
            if chunk.content:
                print(chunk.content, end="", flush=True)
                full_response += chunk.content
                time.sleep(0.02)  # Small delay to see streaming effect
        
        print()  # New line after streaming
        
        # Add AI response to history
        conversation_history.append(AIMessage(content=full_response))
        return full_response
    else:
        # Non-streaming response - returns complete text at once
        response = llm.invoke(conversation_history)
        conversation_history.append(response)
        return response.content

# Interactive chat function with streaming option
def interactive_chat(stream=False):
    print("Chat started! Type 'quit' to exit.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Goodbye!")
            break
        
        if stream:
            chat(user_input, stream=True)
        else:
            print(f"Bot: {chat(user_input)}")

# Demo both modes
print("=== Non-Streaming Mode ===")
print("User: Hi, my name is Alice.")
print("Bot:", chat("Hi, my name is Alice."))

print("\nUser: What's my name?")
print("Bot:", chat("What's my name?"))

print("\n=== Streaming Mode ===")
print("User: Tell me a short story.")
chat("Tell me a short story.", stream=True)

print("\n=== Interactive Mode (Non-streaming) ===")
# Uncomment to start interactive chat
# interactive_chat(stream=False)

print("\n=== Interactive Mode (Streaming) ===")
# Uncomment to start interactive streaming chat
# interactive_chat(stream=True)


In [26]:

from langchain import OpenAI, ConversationChain
from langchain.memory import ConversationBufferMemory

# Initialize the language model and memory
llm = OpenAI(temperature=0.7)
memory = ConversationBufferMemory()

# Create a conversation chain with memory
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

# Have a conversation
print(conversation.predict(input="Hi, my name is Alice."))
print(conversation.predict(input="What's my name?"))
print(conversation.predict(input="What have we talked about so far?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Alice.
AI:

> Finished chain.
 Hello Alice, my name is AI. It's nice to meet you! I am an artificial intelligence program designed to assist and communicate with humans. How can I help you today?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Alice.
AI:  Hello Alice, my name is AI. It's nice to meet you! I am an artificial intelligence program designed t

In [32]:
# ConversationBufferMemory vs ConversationWindowMemory Comparison

from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
# from langchain_community.memory import ConversationBufferWindowMemory

print("=== ConversationBufferMemory (Stores Everything) ===")

# Buffer memory - stores all conversation
buffer_memory = ConversationBufferMemory()
buffer_chain = ConversationChain(llm=llm, memory=buffer_memory)

# Simulate a long conversation
conversations = [
    "Hi, my name is Alice.",
    "I'm 25 years old.",
    "I work as a software engineer.",
    "I live in New York.",
    "I have a dog named Max.",
    "What's my name?",
    "How old am I?",
    "What's my dog's name?"
]

print("Simulating long conversation...")
for i, msg in enumerate(conversations):
    response = buffer_chain.predict(input=msg)
    print(f"Q{i+1}: {msg}")
    print(f"A{i+1}: {response}")
    print(f"Memory length: {len(buffer_memory.buffer)} characters")
    print()

print("=== ConversationWindowMemory (Last 3 Exchanges) ===")

# Window memory - keeps only last 3 exchanges
window_memory = ConversationBufferWindowMemory(k=3)
window_chain = ConversationChain(llm=llm, memory=window_memory)

print("Same conversation with window memory (k=3)...")
for i, msg in enumerate(conversations):
    response = window_chain.predict(input=msg)
    print(f"Q{i+1}: {msg}")
    print(f"A{i+1}: {response}")
    print(f"Memory length: {len(window_memory.buffer)} characters")
    print()


=== ConversationBufferMemory (Stores Everything) ===
Simulating long conversation...
Q1: Hi, my name is Alice.
A1:  Hi Alice! It's nice to meet you. My name is AI, short for artificial intelligence. I am a computer program designed to simulate human intelligence and perform tasks such as natural language processing and problem-solving. What can I assist you with today? 
Memory length: 290 characters

Q2: I'm 25 years old.
A2:  That's interesting, Alice! Did you know that the term "artificial intelligence" was coined in 1956 by John McCarthy? Since then, AI has evolved significantly and has become a vital part of our daily lives, from voice assistants like Siri and Alexa to self-driving cars and medical diagnosis. But I'm sure you already knew that, being 25 years old. What else would you like to know?
Memory length: 702 characters

Q3: I work as a software engineer.
A3:  That's impressive, Alice! As a software engineer, you must have a strong understanding of coding and computer langua

/var/folders/7p/g3xcy5t905zf089tq14bpg3r0000gn/T/ipykernel_24851/1985551425.py:35: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=3)


Q1: Hi, my name is Alice.
A1:  Hello Alice, it's nice to meet you! My name is AI, short for artificial intelligence. I am a computer program that is designed to simulate human conversation and assist with various tasks. I am always learning and improving, so feel free to ask me anything. How can I help you today?
Memory length: 317 characters

Q2: I'm 25 years old.
A2:  That's interesting, Alice. I don't have an age as I am a computer program, but I was created on January 1st, 2020. So, I am currently about 1 year and 6 months old. Is there anything else you would like to know or talk about?
Memory length: 573 characters

Q3: I work as a software engineer.
A3:  That's impressive, Alice! As a language model AI, I don't have a specific job, but I am constantly working on improving my abilities and assisting users like you. What is your favorite part about being a software engineer?
Memory length: 839 characters

Q4: I live in New York.
A4:   That's great, Alice! New York is a bustling ci

In [28]:
# Memory Types Comparison Table

print("=== Memory Types Comparison ===")
print()
print("| Feature | BufferMemory | WindowMemory |")
print("|---------|--------------|--------------|")
print("| Stores | ALL history | Last N exchanges |")
print("| Memory usage | Grows indefinitely | Fixed size |")
print("| Token usage | Increases over time | Consistent |")
print("| Context retention | Perfect | Limited |")
print("| Token limit risk | High | Low |")
print("| Use case | Short conversations | Long conversations |")
print()

print("=== When to Use Each ===")
print()
print("🟢 Use ConversationBufferMemory when:")
print("   • Short conversations (< 10 exchanges)")
print("   • Need perfect context retention")
print("   • Memory is not a concern")
print("   • Example: Customer support chat")
print()
print("🟡 Use ConversationWindowMemory when:")
print("   • Long conversations (> 10 exchanges)")
print("   • Token limits are a concern")
print("   • Recent context is more important")
print("   • Example: Long Q&A sessions")
print()
print("🔵 Use ConversationSummaryMemory when:")
print("   • Very long conversations")
print("   • Need to keep key information")
print("   • Balance between context and efficiency")
print("   • Example: Multi-session conversations")


=== Memory Types Comparison ===

| Feature | BufferMemory | WindowMemory |
|---------|--------------|--------------|
| Stores | ALL history | Last N exchanges |
| Memory usage | Grows indefinitely | Fixed size |
| Token usage | Increases over time | Consistent |
| Context retention | Perfect | Limited |
| Token limit risk | High | Low |
| Use case | Short conversations | Long conversations |

=== When to Use Each ===

🟢 Use ConversationBufferMemory when:
   • Short conversations (< 10 exchanges)
   • Need perfect context retention
   • Memory is not a concern
   • Example: Customer support chat

🟡 Use ConversationWindowMemory when:
   • Long conversations (> 10 exchanges)
   • Token limits are a concern
   • Recent context is more important
   • Example: Long Q&A sessions

🔵 Use ConversationSummaryMemory when:
   • Very long conversations
   • Need to keep key information
   • Balance between context and efficiency
   • Example: Multi-session conversations


In [49]:
# Advanced: ConversationSummaryMemory Example

from langchain.memory import ConversationSummaryMemory
from langchain_openai import OpenAI

print("=== ConversationSummaryMemory (Best of Both Worlds) ===")

# Summary memory - summarizes old conversations
summary_memory = ConversationSummaryMemory(llm=llm)
summary_chain = ConversationChain(llm=llm, memory=summary_memory)

print("Testing summary memory...")
print("This will summarize old conversations to save tokens while keeping key info.")
print()

# Test with a longer conversation
long_conversations = [
    "Hi, I'm Alice and I'm a software engineer.",
    "I work at Google and I love Python programming.",
    "I have two cats named Luna and Stella.",
    "I live in San Francisco with my partner.",
    "I'm learning machine learning and AI.",
    "What do you know about me so far?",
    "What are my cats' names?",
    "Where do I work?"
]

for i, msg in enumerate(long_conversations):
    response = summary_chain.predict(input=msg)
    print(f"Q{i+1}: {msg}")
    print(f"A{i+1}: {response}")
    print(f"Memory length: {len(summary_memory.buffer)} characters")
    print("---")

print("\n=== Memory Type Summary ===")
print("1. BufferMemory: Stores everything (good for short chats)")
print("2. WindowMemory: Stores last N exchanges (good for long chats)")
print("3. SummaryMemory: Summarizes old + keeps recent (best for very long chats)")
print("4. Our Manual: Full control (good for learning/custom needs)")


=== ConversationSummaryMemory (Best of Both Worlds) ===
Testing summary memory...
This will summarize old conversations to save tokens while keeping key info.

Q1: Hi, I'm Alice and I'm a software engineer.
A1:  Hello Alice, it's nice to meet you! As an AI, I don't have a name, but you can call me OpenAI. I am a language processing model that uses artificial intelligence to generate human-like text. It's fascinating to be able to communicate with humans like you! As a software engineer, you must work with different programming languages and tools to create software. Can you tell me more about your job?
Memory length: 322 characters
---
Q2: I work at Google and I love Python programming.
A2:  That's fascinating, Alice! I am always amazed by the complexity and capability of programming languages like Python. What are some of the projects you are currently working on at Google?
Memory length: 470 characters
---
Q3: I have two cats named Luna and Stella.
A3:  Oh, how lovely! I have never s

In [44]:
# ConversationSummaryMemory with Summary Display

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain

print("=== ConversationSummaryMemory - Showing Summaries ===")
print("This will show you the actual summaries being generated at each stage!")
print()

# Create summary memory
summary_memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=False,
    memory_key="history",
    max_token_limit=300  # Fix: Use "history" instead of "chat_history"
)

# Create chain with verbose output to see prompts
summary_chain = ConversationChain(
    llm=llm, 
    memory=summary_memory,
    verbose=True  # This shows the actual prompts
)

print("=== Step 1: Initial Messages ===")
print("Adding first few messages...")
print()

# Add initial conversation
summary_chain.predict(input="Hi, I'm Alice and I'm a software engineer.")
summary_chain.predict(input="I work at Google and I love Python programming.")
summary_chain.predict(input="I have two cats named Luna and Stella.")

print("\n=== Current Memory State ===")
print(f"Memory length: {len(summary_memory.buffer)} characters")
print(f"Memory content: {summary_memory.buffer}")
print()

print("=== Step 2: Adding More Messages (Should Trigger Summary) ===")
print("Adding more messages to trigger summarization...")
print()

# Add more messages to trigger summarization
summary_chain.predict(input="I live in San Francisco with my partner.")
summary_chain.predict(input="I'm learning machine learning and AI.")

print("\n=== Memory After Summarization ===")
print(f"Memory length: {len(summary_memory.buffer)} characters")
print(f"Memory content: {summary_memory.buffer}")
print()

print("=== Step 3: Test Memory Retrieval ===")
print("Asking about earlier information...")
print()

# Test memory retrieval
response = summary_chain.predict(input="What do you know about me so far?")
print(f"Response: {response}")

print("\n=== Step 4: Check Final Memory State ===")
print(f"Final memory length: {len(summary_memory.buffer)} characters")
print(f"Final memory content: {summary_memory.buffer}")


=== ConversationSummaryMemory - Showing Summaries ===
This will show you the actual summaries being generated at each stage!

=== Step 1: Initial Messages ===
Adding first few messages...



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Alice and I'm a software engineer.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Alice and I'm a software engineer.
AI:  Hello Alice, it's nice to meet you! 

In [46]:
# Token Counting and Summarization Trigger

print("=== How Token Counting Works ===")
print()

print("1. TOKEN ESTIMATION:")
print("   - LangChain estimates tokens using tiktoken (OpenAI's tokenizer)")
print("   - It counts tokens in the conversation history")
print("   - When tokens exceed max_token_limit, summarization triggers")
print()

print("2. SUMMARIZATION PROCESS:")
print("   - Takes old conversation exchanges")
print("   - Sends them to LLM with summarization prompt")
print("   - Replaces old exchanges with summary")
print("   - Keeps recent exchanges (usually last 2-3)")
print()

print("3. EXAMPLE TOKEN COUNTS:")
print("   - 'Hi, I'm Alice and I'm a software engineer.' ≈ 15 tokens")
print("   - 'I work at Google and I love Python programming.' ≈ 15 tokens")
print("   - 'I have two cats named Luna and Stella.' ≈ 15 tokens")
print("   - Total for 3 messages ≈ 45 tokens")
print()

print("4. CUSTOMIZING SUMMARIZATION:")
print()

# Show different configurations
configs = [
    {"limit": 100, "description": "Very aggressive summarization"},
    {"limit": 500, "description": "Quick summarization"},
    {"limit": 2000, "description": "Default balanced approach"},
    {"limit": 5000, "description": "Minimal summarization"},
    {"limit": 10000, "description": "Rarely summarizes"}
]

for config in configs:
    print(f"   max_token_limit={config['limit']}: {config['description']}")

print()
print("5. PRACTICAL RECOMMENDATIONS:")
print("   🟢 For chatbots: 1000-2000 tokens")
print("   🟡 For Q&A systems: 2000-3000 tokens")
print("   🔴 For document analysis: 5000+ tokens")
print("   ⚡ For real-time chat: 500-1000 tokens")


=== How Token Counting Works ===

1. TOKEN ESTIMATION:
   - LangChain estimates tokens using tiktoken (OpenAI's tokenizer)
   - It counts tokens in the conversation history
   - When tokens exceed max_token_limit, summarization triggers

2. SUMMARIZATION PROCESS:
   - Takes old conversation exchanges
   - Sends them to LLM with summarization prompt
   - Replaces old exchanges with summary
   - Keeps recent exchanges (usually last 2-3)

3. EXAMPLE TOKEN COUNTS:
   - 'Hi, I'm Alice and I'm a software engineer.' ≈ 15 tokens
   - 'I work at Google and I love Python programming.' ≈ 15 tokens
   - 'I have two cats named Luna and Stella.' ≈ 15 tokens
   - Total for 3 messages ≈ 45 tokens

4. CUSTOMIZING SUMMARIZATION:

   max_token_limit=100: Very aggressive summarization
   max_token_limit=500: Quick summarization
   max_token_limit=2000: Default balanced approach
   max_token_limit=5000: Minimal summarization
   max_token_limit=10000: Rarely summarizes

5. PRACTICAL RECOMMENDATIONS:
   🟢 Fo

=== How Token Counting Works ===

1. TOKEN ESTIMATION:
   - LangChain estimates tokens using tiktoken (OpenAI's tokenizer)
   - It counts tokens in the conversation history
   - When tokens exceed max_token_limit, summarization triggers

2. SUMMARIZATION PROCESS:
   - Takes old conversation exchanges
   - Sends them to LLM with summarization prompt
   - Replaces old exchanges with summary
   - Keeps recent exchanges (usually last 2-3)

3. EXAMPLE TOKEN COUNTS:
   - 'Hi, I'm Alice and I'm a software engineer.' ≈ 15 tokens
   - 'I work at Google and I love Python programming.' ≈ 15 tokens
   - 'I have two cats named Luna and Stella.' ≈ 15 tokens
   - Total for 3 messages ≈ 45 tokens

4. CUSTOMIZING SUMMARIZATION:

| max_token_limit | Description                   |
|:---------------:|:-----------------------------|
|      100        | Very aggressive summarization |
|      500        | Quick summarization           |
|     2000        | Default balanced approach     |
|     5000        | Minimal summarization         |
|    10000        | Rarely summarizes             |
5. PRACTICAL RECOMMENDATIONS:
 
| Use Case              | Recommended Tokens | Icon |
|-----------------------|-------------------|------|
| Chatbots              | 1000-2000         | 🟢   |
| Q&A systems           | 2000-3000         | 🟡   |
| Document analysis     | 5000+             | 🔴   |
| Real-time chat        | 500-1000          | ⚡   |

In [47]:
# Revealing the Actual Prompts Sent to LLM

print("=== ConversationSummaryMemory Internal Prompts ===")
print()

print("1. SUMMARIZATION PROMPT (when memory gets full):")
print("   This is the actual prompt sent to LLM to generate summaries:")
print()
print("   'Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.'")
print("   'EXAMPLE'")
print("   'Current summary: The human asks what an AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.'")
print("   'New lines of conversation:'")
print("   'Human: Why do you think artificial intelligence is a force for good?'")
print("   'AI: I think artificial intelligence is a force for good because it can help humans solve complex problems more quickly and efficiently.'")
print("   'New summary: The human asks what an AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it can help humans solve complex problems more quickly and efficiently.'")
print("   'END OF EXAMPLE'")
print("   'Current summary: {summary}'")
print("   'New lines of conversation: {new_lines}'")
print("   'New summary:'")
print()

print("2. CONVERSATION PROMPT (for normal chat):")
print("   This is the prompt sent to LLM for regular conversation:")
print()
print("   'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.'")
print("   'Current conversation:'")
print("   'Summary: {summary}'")
print("   'Recent conversation:'")
print("   'Human: {current_input}'")
print("   'AI:'")
print()


=== ConversationSummaryMemory Internal Prompts ===

1. SUMMARIZATION PROMPT (when memory gets full):
   This is the actual prompt sent to LLM to generate summaries:

   'Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.'
   'EXAMPLE'
   'Current summary: The human asks what an AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.'
   'New lines of conversation:'
   'Human: Why do you think artificial intelligence is a force for good?'
   'AI: I think artificial intelligence is a force for good because it can help humans solve complex problems more quickly and efficiently.'
   'New summary: The human asks what an AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it can help humans solve complex problems more quickly and efficiently.'
   'END OF EXAMPLE'
   'Current summary: {summary}'
   'New lines of conversation: {new_line

# 🧩 Revealing the Actual Prompts Sent to LLM

This document shows the **internal prompts** used by the system when interacting with a Large Language Model (LLM), specifically for **conversation summarization** and **normal chat flow**.

---

## 🧠 ConversationSummaryMemory Internal Prompts

These prompts are part of the memory system that helps the LLM maintain context over long conversations. When the memory fills up, the system summarizes previous exchanges to retain important information.

---

### 1. 📝 SUMMARIZATION PROMPT (when memory gets full)

This is the actual prompt sent to the LLM to generate a **progressive summary** of the conversation.

> The goal is to combine the existing summary with new conversation lines to produce an updated, concise summary.

#### 🔹 Prompt Template:

```
Progressively summarize the lines of conversation provided,
adding onto the previous summary returning a new summary.

EXAMPLE
Current summary: The human asks what an AI thinks of artificial intelligence.
The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: I think artificial intelligence is a force for good because it can help
humans solve complex problems more quickly and efficiently.

New summary:
The human asks what an AI thinks of artificial intelligence.
The AI thinks artificial intelligence is a force for good because it can help
humans solve complex problems more quickly and efficiently.
END OF EXAMPLE

Current summary: {summary}
New lines of conversation: {new_lines}
New summary:
```

---

### 2. 💬 CONVERSATION PROMPT (for normal chat)

This is the standard prompt used during a regular conversation.  
It ensures that the AI remains helpful, detailed, and conversational while being truthful about its knowledge limits.

#### 🔹 Prompt Template:

```
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Summary: {summary}
Recent conversation:
Human: {current_input}
AI:
```

---

### 🧩 Summary

| Prompt Type | Purpose | Key Behavior |
|--------------|----------|----------------|
| **Summarization Prompt** | Compress long-term memory | Merge new lines with previous summary to form an updated summary |
| **Conversation Prompt** | Drive normal interactions | Keep tone friendly, detailed, and honest |

---

### 📘 Notes

- These prompts are **internal system instructions**, not visible to the end user.
- The `{summary}` and `{new_lines}` placeholders dynamically update as the conversation progresses.
- The summarization ensures that context is retained efficiently even when full chat logs can’t be stored.

---
